In [1]:
from pymilvus import connections, Collection, utility
from pymilvus import utility
import google.generativeai as genai
from local_settings import API_KEY,uri,token
from pymilvus import MilvusClient
from helper_func import create_prompt, process_data

connections.connect(uri=uri,token=token)
milvus_client = MilvusClient(uri=uri,token=token)

collection_name = "my_rag_collection2"

In [2]:

collection = Collection(collection_name)
collection.num_entities 

21326

In [3]:
utility.list_collections()

['my_rag_collection2']

In [4]:
res = collection.query(expr="", output_fields=["count(*)"])
print(res)

[{'count(*)': 21326}]


In [5]:
embedding_model = 'models/text-embedding-004'
genai.configure(api_key=API_KEY)
def embed_fn(title, text):
  return genai.embed_content(model=embedding_model,
                             content=text,
                             task_type="retrieval_document",
                             title=title)["embedding"]

In [6]:
question = "What does do you do?"

search_res = milvus_client.search(
    collection_name=collection_name,
    data=[
        embed_fn(question,question)
    ],  # Use the `emb_text` function to convert the question to an embedding vector
    limit=7,  # Return top 3 results
    search_params={"metric_type": "COSINE",  "params": {"level": 2}},  # Inner product distance
    output_fields=["text"],  # Return the text field
)

In [7]:
retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
retrieved_lines_with_distances

[(['Can You Provide A Sample Answer For Everyone To See? ',
   'Here is a sample answer . Authors simply answer the questions posed by those asking them , to the best of our ability . The answers must be at least 50 words in length so that there is information conveyed rather than just yes or no responses . We are not allowed to copy answers from other websites or repeat answers verbatim . '],
  0.575653076171875),
 (['Why Can ',
   "Why can ? A simple question , but yet , so complex ! Why can someone do this or why can someone do that ? I have often pondered for hours to come up with the answer . And I believe , after years of thought-provoking consultation with friends and relatives ... I have the answer to the question `` Why can ? '' The answer ? Why not ? "],
  0.5728439092636108),
 (['What Does A Health Insurance Broker Do? ',
   "What does n't a health insurance broker . To put it simply we interact with all of the different elements in health insurance , clients , carriers , an

In [19]:

querry_data= process_data(retrieved_lines_with_distances)
print(querry_data)

Q: Can You Provide A Sample Answer For Everyone To See?
A: Here is a sample answer . Authors simply answer the questions posed by those asking them , to the best of our ability . The answers must be at least 50 words in length so that there is information conveyed rather than just yes or no responses . We are not allowed to copy answers from other websites or repeat answers verbatim .

Q: Why Can
A: Why can ? A simple question , but yet , so complex ! Why can someone do this or why can someone do that ? I have often pondered for hours to come up with the answer . And I believe , after years of thought-provoking consultation with friends and relatives ... I have the answer to the question `` Why can ? '' The answer ? Why not ?

Q: What Does A Health Insurance Broker Do?
A: What does n't a health insurance broker . To put it simply we interact with all of the different elements in health insurance , clients , carriers , and doctors . We are an advocate for our client first . We will eval

In [20]:
%%time
testing_question=""
prompt=create_prompt(querry_data,testing_question)
print(prompt)


Task: Your are a Insurance assistance, your job is based on a conversation of a costumer and a insurance expert, 
answer insurance-related questions accurately and provide some short explanation.
Instructions:
Read the context carefully.
Consider key points, concepts, and details within the document.
Generate questions that seek information or clarification about the document's content.
Ensure that the questions are grammatically correct and understandable.
Give the answer directly, do not repeat the question or add any unrelated text
ONLY answers question related to the function of the application or insurance
If the question is not related to the context or there is no answer in the context, tell the users the question is not related to insurance or there is no answer in your database
Context:
Q: Can You Provide A Sample Answer For Everyone To See?
A: Here is a sample answer . Authors simply answer the questions posed by those asking them , to the best of our ability . The answers m

In [21]:
%%time
LLM_model = genai.GenerativeModel('gemini-pro')
response = LLM_model.generate_content(prompt)

CPU times: total: 0 ns
Wall time: 2.32 s


In [22]:
print( response.text.replace("JSON", "").replace("```",""))

Question is not related to insurance
